# Thermokarst features as indicators of permafrost thaw in arctic landscapes 

Thermokarst features are topographic depressions created when permafrost thaws. Identifying thermokarst features in freely available satellite imagery and tracking changes in these features over time could give insights into landscape change dyanmics in a rapidly warming arctic. 

In the MVP for this project, we attempt to use a supervised machine learning model to classify thermokarst features in arctic landscapes. Our main objectives are: 
   1) Obtain RGB satellite imagery (Planet composite images, or maybe Sentinal or Landsat). 
   
   2) Attempt to create training data using an unsupervised classification method. If this fails, we will use a supervised classification method.  
   
   3) Train a random forest regression model to identify thermokarst features. 
   
   4) Create a data training mask and see how that improves model performance. 

For the final project, we will continue to build the training dataset to improve this model. We will also improve our classification model by integrating DEM and NDVI/NDWI information in the classification model. We will demonstrate the ability to calculate/identify thermokarst expansion using images classified under this model for different time periods, and will consider parallelizing parts of our workflow in order to allow for faster processing times of large amounts of spatial data. As time allows, we may investigate using output maps of landscape change to predict future change under warming scenarios.  

## Resources and references used

### Spatial Science tutorials
[Tutorial: Land classification with machine learning](https://geohackweek.github.io/machine-learning/03-landclass/) - includes random forest and SVM models 

[Tutorial: landcover classification of satellite imagery using CNNs](https://towardsdatascience.com/land-cover-classification-of-satellite-imagery-using-convolutional-neural-networks-91b5bb7fe808) 

[Tutorial: Image processing with Python: Unsupervised Learning for image segementation](https://towardsdatascience.com/image-processing-with-python-unsupervised-learning-for-image-segmentation-90ebd23d91a4)

### References
[Machine learning in modelling land-use and land cover-change (LULCC): Current status, challenges and prospects](https://www.sciencedirect.com/science/article/abs/pii/S0048969722006519#f0050)


## Library Import

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import os 

#we will use RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

## Raster to mosaic files

In [ ]:
import tarfile # for reading in the tar files 

# rasters to mosaic 
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path

In [ ]:
# unzip tar file
# open file
file = tarfile.open("./Data/LC08_L1TP_071011_20220812_20220824_02_T1.tar")
  
# extracting file
file.extractall('./Data/LC08_L1TP_071011_20220812_20220824_02_T1')
  
file.close()

In [ ]:
# rasters to mosaic file
#https://medium.com/spatial-data-science/how-to-mosaic-merge-raster-data-in-python-fb18e44f3c8

#physically moved the bands that I wanted into their own folder - fix that later! 
path = Path('./Data/LC08_L1TP_071011_20220812_20220824_02_T1/Bands/')
Path('output').mkdir(parents=True, exist_ok=True)
output_path = 'output/LC08_L1TP_071011_20220812_20220824_02_T1_mosaic_output.tif'

raster_files = list(path.iterdir())
raster_to_mosiac = []

# append on all of the rasters that we want
for p in raster_files:
    raster = rio.open(p)
    raster_to_mosiac.append(raster)

# merge 
mosaic, output = merge(raster_to_mosiac)

# Copy metadata
output_meta = raster.meta.copy()
output_meta.update(
    {"driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": output,
    }
)


In [ ]:
with rio.open(output_path,"w", **output_meta) as m:
    m.write(mosaic)

## Unsupervised land classification python

In [ ]:
conda install -c conda-forge gdal

In [ ]:
# the libraries
from sklearn.cluster import KMeans
import gdal
import numpy as np

In [ ]:
# read in image to classify with gdal
naip_fn = './output/LC08_L1TP_071011_20220812_20220824_02_T1_mosaic_output.tif'
driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)
nbands = naip_ds.RasterCount